# Securing an atoti session

Securing a session comes in two parts:
1. Implementing an authentication mechanism to secure access to the session
2. Restricting access of modules or data access by users based on the roles granted

With the [Atoti+ plugin](https://docs.atoti.io/latest/atoti_plus.html), atoti supports multiple [authentication mechanisms](https://docs.atoti.io/latest/lib/atoti/atoti.config.authentication.html) to cater to the needs of our end users. This notebook puts together 4 different ways we could authenticate users in atoti, so that you can compare the differences in the implementation.  

You could alternatively focus on the individual implementation in the following notebooks:
- [01-Basic-authentication.ipynb](./01-Basic-authentication.ipynb)
- [02-OIDC-Auth0.ipynb](./02-OIDC-Auth0.ipynb)
- [03-OIDC-Google.ipynb](./03-OIDC-Google.ipynb)
- [04-LDAP.ipynb](./03-OIDC-Google.ipynb)

We will explore security with Atoti+ using the [Top 50 Fast Food](https://www.kaggle.com/datasets/stetsondone/top50fastfood) dataset from Kaggle, combined with its parent company information sourced from the internet.  

__Note__:  

The notebook is structured in this order:
1. Authentication setup during session instantiation
2. Create BI analytics platform with atoti Community Edition
3. Users and roles management with Atoti+

Mainly, we look at the roles last because we need to know the names of the tables and columns which we want to impose restrictions on.  
Thereafter, we can create the roles with restrictions.

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=security-implementation" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/discover.png" alt="Try atoti"></a></div>

In [1]:
import os

import atoti as tt
import ipywidgets as widgets
import pandas as pd
from atoti_plus import UserServiceClient

In [2]:
AUTH_BASIC = "BASIC"
AUTH_OIDC_GOOGLE = "OIDC - Google"
AUTH_OIDC_AUTH0 = "OIDC - Auth0"
AUTH_LDAP = "LDAP"

## 1. Authentication mechanism selection

Assuming you have the corresponding authentication provider configured with the users (and roles where applicable), select the mode of authentication you would like to test.  

__NOTE:__ Restart the session if the mode of authentication is changed. Session will be recreated and consequently, data has to be reloaded and the cube has to be recreated as well.

In [3]:
auth_mode = widgets.RadioButtons(
    options=[AUTH_BASIC, AUTH_OIDC_GOOGLE, AUTH_OIDC_AUTH0, AUTH_LDAP],
    value=AUTH_BASIC,
    description="Mode of authentication:",
    disabled=False,
    style={"description_width": "initial"},
)

display(auth_mode)

RadioButtons(description='Mode of authentication:', options=('BASIC', 'OIDC - Google', 'OIDC - Auth0', 'LDAP')…

### 1.1. Authentication setup in atoti  

We demonstrate below how we configure the authentication parameter of [`atoti.Session`](https://docs.atoti.io/latest/lib/atoti/atoti.session.html#atoti.Session.__init__) for different authentication mechanisms.  

In [4]:
def get_auth_mode():
    print("Mode of auth:", auth_mode.value)

    if auth_mode.value == AUTH_BASIC:
        # https://docs.atoti.io/latest/lib/atoti/atoti.config.authentication.basic_authentication_config.html
        # The realm value (case-sensitive), in combination with the canonical root URL of the server being accessed, defines the protection space.
        authentication = tt.BasicAuthenticationConfig(realm="atoti Realm")

    elif auth_mode.value == AUTH_OIDC_AUTH0:
        # https://docs.atoti.io/latest/lib/atoti/atoti.config.authentication.oidc_config.html
        authentication = tt.OidcConfig(
            provider_id="auth0",
            issuer_url="https://dev-5m2svhd0.us.auth0.com/",
            client_id=os.environ["AUTH0_CLIENT_ID"],
            client_secret=os.environ["AUTH0_CLIENT_SECRET"],
            name_claim="name",
            scopes=["email", "profile", "username", "roles"],
            roles_claims=[
                "https://dev-5m2svhd0:us:auth0:com/roles",
            ],
        )

    elif auth_mode.value == AUTH_OIDC_GOOGLE:
        # https://docs.atoti.io/latest/lib/atoti/atoti.config.authentication.oidc_config.html
        authentication = tt.OidcConfig(
            provider_id="google",
            issuer_url="https://accounts.google.com",
            client_id=os.environ["GOOGLE_CLIENT_ID"],
            client_secret=os.environ["GOOGLE_CLIENT_SECRET"],
            scopes=["https://www.googleapis.com/auth/userinfo.email"],
            name_claim="email",
        )
    elif auth_mode.value == AUTH_LDAP:
        authentication = tt.LdapConfig(
            url="ldap://localhost:10389",
            base_dn="dc=example,dc=com",
            user_search_base="ou=people",
            group_search_base="ou=roles",
        )

    return authentication

Client ids and secrets in the case of OIDC should be kept private. As suggested in the [documentation](https://docs.atoti.io/latest/how_tos/security/secure_a_session.html#Configuring-the-authentication-mechanism), connection details can be read from environment variables for improved security.  

#### 1.1.1. [OIDC - Auth0 only] Authentication config mapping against Auth0 setup  

In Auth0, navigate to Actions > Library and click on "Build Custom" button to create a trigger for the `Login / Post Login` as follows (update namespace according to your environment):

```
exports.onExecutePostLogin = async (event, api) => {
  const namespace = 'https://dev-5m2svhd0.us.auth0.com';
  if (event.authorization) {
    api.idToken.setCustomClaim(`${namespace}/roles`, event.authorization.roles);
    api.accessToken.setCustomClaim(`${namespace}/roles`, event.authorization.roles);
  }
};
```

Thereafter, refer to the following for the setting mapping against Auth0:
<img src="https://data.atoti.io/notebooks/security/img/auth0-setup.png" width="80%" />  

`name_claim` could be email or name, depending on what you would like to see reflected in the application:  

<img src="https://data.atoti.io/notebooks/security/img/displayed_id.png" width="80%" />  

#### 1.1.2. [OIDC - Google only] Authentication config mapping against Google setup

Firstly, we have to configure the _OAuth consent screen_ and registered our app. Remember to set the "User type" to _Internal_, so that our app is limited to Google Workspace users within our organisation.  

<img src="https://data.atoti.io/notebooks/security/img/google-oauth-consent-screen.png" width="70%" />

Proceed to create a credential of type "OAuth client IDs" under __APIs & Services > Credentials__ in Google Cloud.  
Thereafter, refer to the following for the setting mapping against google:  
<img src="https://data.atoti.io/notebooks/security/img/google-setup.png" width="70%" />

#### 1.1.3. [LDAP] Authentication config mapping

In our example, we used the [Apache DS](https://directory.apache.org/apacheds/) to configure our LDAP server.

Use `user_search_base` to search the group where the user id (UID) resides under.  
Use `group_search_base` to search the group where the common names (CN) resides under.  

<img src="https://data.atoti.io/notebooks/security/img/ldap-setup.png" width="70%" />  

__All users should have the role equivalent to `ROLE_USER` to be able to access atoti web application.__

### 1.2. Session instantiation with authentication 

For OIDC setup, it is important to fix the port to configure some URLs for callbacks.  

<img src="https://data.atoti.io/notebooks/security/img/callbacks.png" width="70%" />  

We use the [provider id](https://docs.atoti.io/latest/lib/atoti/atoti.config.authentication.oidc_config.html#atoti.OidcConfig.provider_id) (e.g. google or auth0) to build the redirect URL:

`f"{session_url}/login/oauth2/code/{provider_id}"`  

In any case, setting up firewall is part of the security setup and this would require a fixed port for the application too.

In [5]:
session = tt.Session(
    port=10011, authentication=get_auth_mode(), user_content_storage="./content"
)

Mode of auth: BASIC


From [atoti v0.7.2](https://docs.atoti.io/latest/releases/0.7.2.html#added) onwards, atoti uses [UserServiceClient](https://docs.atoti.io/latest/lib/atoti-plus/atoti_plus.user_service_client.user_service_client.html#atoti_plus.UserServiceClient) to manage the dynamic aspects of the security configuration.  
With the client, multiple sessions can be configured with the same security configuration stored on the _user content storage_. The security configuration covers the roles and restrictions.

In [6]:
user_service_client = UserServiceClient.from_session(session)

## 2. Create BI analytics platform with atoti Community Edition

Once the session is created, we can proceed with the usual data loading into atoti table, cube and measures creation.  
Remember to re-execute these cells if you have changed the mode of authentiction.

### 2.1 Table creation

Although we can [`create table`](https://docs.atoti.io/latest/lib/atoti/atoti.session.html#atoti.Session.create_table) before loading data in, we used `read_csv` in our example to create and load data into the atoti tables.

In [7]:
base_tbl = session.read_csv(
    "s3://data.atoti.io/notebooks/security/data/parent_co.csv",
    table_name="parent_co",
    keys=["company", "parent_company"],
    process_quotes=True,
)
base_tbl.head()

,,name
parent_company,company,
Inspire Brands,arbys,Arby's
"Domino's Pizza, Inc.",dominos,Domino's
"Papa Murphy's Holdings, Inc.",papa_murphys,Papa Murphy's
Focus Brands,auntie_annes,Auntie Anne's
Panda Restaurant Group,panda_express,Panda Express


In [8]:
enrichment_tbl = session.read_csv(
    "s3://data.atoti.io/notebooks/security/data/top_50_fast_food_US.csv",
    table_name="top_50",
    keys=["company"],
)
enrichment_tbl.head()

,category,sales_in_millions_2019,sales_per_unit_thousands_2019,franchised_units_2019,company_owned_units_2019,total_units_2019,unit_change_from_2018
company,,,,,,,
popeyes_chicken,chicken,3750,1541,2458,41,2499,131
del_taco,global,850,1554,296,300,596,16
jack_in_the_box,burger,3505,1565,2106,137,2243,6
tim_hortons,sandwich,840,1165,715,0,715,-12
mcdonalds,burger,40413,2912,13154,692,13846,-66


In [9]:
base_tbl.join(enrichment_tbl)

### 2.2. Cube creation

In [10]:
cube = session.create_cube(base_tbl, "Fast food analysis")

### 2.3 Measures creation

In [11]:
h, l, m = cube.hierarchies, cube.levels, cube.measures

In [12]:
m["sales_in_millions"] = tt.agg.sum(enrichment_tbl["sales_in_millions_2019"])
m["sales_per_unit_thousands"] = tt.agg.sum(
    enrichment_tbl["sales_per_unit_thousands_2019"]
)
m["franchised_units"] = tt.agg.sum(enrichment_tbl["franchised_units_2019"])
m["company_owned_units"] = tt.agg.sum(enrichment_tbl["company_owned_units_2019"])
m["total_units"] = tt.agg.sum(enrichment_tbl["total_units_2019"])
m["unit_change_from_2018"] = tt.agg.sum(enrichment_tbl["unit_change_from_2018"])

In [13]:
m["% franchised"] = m["franchised_units"] / m["total_units"]
m["% franchised"].formatter = "DOUBLE[0.00%]"

## 3. Manage users and roles with Atoti+

### 3.1. [Basic Authentication only] User creation  

Users' information is retrieved from authentication provider if one is used. However, in the case of basic authentication, users are created in atoti.
Making use of the [BasicSecurity](https://docs.atoti.io/latest/lib/atoti-plus/atoti_plus.security.html#atoti_plus.security.BasicSecurity) module, we create an atoti administrator and a generic atoti user as shown below:

In [14]:
if auth_mode.value == AUTH_BASIC:
    user_service_client.basic.create_user("atoti_admin", password="atoti_pwd")
    user_service_client.basic.create_user("atoti_user", password="atoti_pwd")

We cherry-picked two parent companies - Inspire Brands and Restaurant Brands International LLC to demonstrate the roles and access management.  
We will create the users based on the list available on [basic_user_pwd.csv](https://data.atoti.io/notebooks/security/users/basic_user_pwd.csv).

In [15]:
if auth_mode.value == AUTH_BASIC:
    users_df = pd.read_csv(
        "https://data.atoti.io/notebooks/security/users/basic_user_pwd.csv"
    )
    for row in users_df.to_dict(orient="records"):
        user = user_service_client.basic.create_user(
            row["User"], password=row["Password"]
        )
        print(f"Create user: {user} - role(s): {user.roles}")

Create user: <atoti_plus.user_service_client.user.User object at 0x000001E2444B7C70> - role(s): ['ROLE_USER']
Create user: <atoti_plus.user_service_client.user.User object at 0x000001E2444D8250> - role(s): ['ROLE_USER']
Create user: <atoti_plus.user_service_client.user.User object at 0x000001E2444B7C70> - role(s): ['ROLE_USER']
Create user: <atoti_plus.user_service_client.user.User object at 0x000001E2444D81C0> - role(s): ['ROLE_USER']
Create user: <atoti_plus.user_service_client.user.User object at 0x000001E2444B7C70> - role(s): ['ROLE_USER']
Create user: <atoti_plus.user_service_client.user.User object at 0x000001E2444D80D0> - role(s): ['ROLE_USER']


The system reserved role `ROLE_USER` is automatically assigned to the created users. This meant that these users will all be able to access all the web application and all the available data.  
We can, however, restrict access for the users by assigning them roles with restricted access. 

### 3.2. Roles management in atoti

#### 3.2.1. atoti reserved roles  

The below roles are reserved in atoti and should not be altered by users:
- ROLE_ADMIN: able to access all objects in the web application
- ROLE_USER: able to access all data by default. Access to objects such as dashboards, folders, widgets etc is only upon sharing access granted to role.
- ROLE_SHARE: able to share objects such as dashboards, folders, widgets and filters.  

__All users, including the administrator, require the role *ROLE\_USER* to be able to access the atoti web application.__

Let's assume both users, *atoti\_admin* and *atoti\_user*, have been granted the role __ROLE_USER__ and *atoti\_admin* is also granted the role __ROLE_ADMIN__.  
While both *atoti\_admin* and *atoti\_user* are able to access all data, *atoti\_admin* is able to access all objects such as folders and dashboards.  
*atoti\_user* is able to access only the objects created by the user him/herself. Objects created by other users can only be access upon shared access granted.  
Both *atoti\_admin* and *atoti\_user* will not be able to share objects (via the "Share" icon as shown below) unless granted the role __ROLE_SHARE__.  

<img src="https://data.atoti.io/notebooks/security/img/sharing.png" width="70%"/>

#### 3.2.2. Role creation with restrictions  

Data restriction is based on users' requirement. In our use case, we assumed two groups of users with data access limited to those of their parent company:
- users belonging to parent company _Inspire Brands_
- users belonging to parent company _Restaurant Brands International Inc._

Therefore, we will create two roles to apply the restrictions based on the `parent_company` column from the `parent_co` table.  
We will define key that is a tuple, consisting of the name of the table and its column, along with the restricted values imposed on it. 

__NOTE:__  
- We can skip role creation if there are no restrictions imposed on the role. 
- The value provided under the restrictions is cap-sensitive.

In [16]:
role_inspire = user_service_client.create_role(
    "ATOTI_ROLE_INSPIRE",
    restrictions={("parent_co", "parent_company"): ["Inspire Brands"]},
)

role_restaurant = user_service_client.create_role(
    "ATOTI_ROLE_RESTAURANT",
    restrictions={
        ("parent_co", "parent_company"): ["Restaurant Brands International Inc."]
    },
)

#### 3.2.3. Restricted access from combination of roles

Multiple roles can be assigned to the same user. To demonstrate how the access will change when this happens, we create some other roles that restrict data access by the restaurant category, i.e. column `category` from the table `top_50`.

In [17]:
role_team_burger = user_service_client.create_role(
    "ATOTI_ROLE_BURGER", restrictions={("top_50", "category"): ["burger"]}
)

role_team_sandwich = user_service_client.create_role(
    "ATOTI_ROLE_SANDWICH", restrictions={("top_50", "category"): ["sandwich"]}
)

role_team_snack = user_service_client.create_role(
    "ATOTI_ROLE_SNACK", restrictions={("top_50", "category"): ["snack"]}
)

When combined with the restricted role on the `parent_company`, user's access will be further restricted to based on the restriction of the added role.  

For instance, users who are assigned the role __ATOTI_ROLE_BURGER__ will be able to access all the data under _burger_ category restaurants, regardless of the parent companies.  

However, when the same user is also granted the role __ATOTI_ROLE_INSPIRE__, then the user can only access data of restaurants under parent company _Inspire Brands_ that is of category _burger_. 

### 3.3. [Basic Authentication only] Assigning user roles to user (Good reference for roles setup in authentication providers)  

We can grant atoti roles directly to users created in atoti without having to perform role mappings like in OIDC.
But, we can easily demonstrate how roles can be assigned to the users to restrict data access. It can be easily inferred and applied on the authentication providers when granting roles.

#### 3.3.1 ROLE_ADMIN, ROLE_USER and ROLE_SHARE

While both *atoti\_admin* and *atoti\_user* are able to access all data, *atoti\_admin* is able to access all objects such as folders and dashboards.  
*atoti\_user* is able to access only the objects created by the user him/herself. Objects created by other users can only be access upon shared access granted.  

Both *atoti\_admin* and *atoti\_user* will not be able to share objects unless granted the role __ROLE_SHARE__.  
We will grant *atoti\_user* the ability to share the objects he/she created for demonstrative purpose.  

<img src="https://data.atoti.io/notebooks/security/img/sharing.png" width="70%"/>

In [18]:
if auth_mode.value == AUTH_BASIC:
    user_service_client.individual_roles["atoti_admin"].add("ROLE_ADMIN")
    user_service_client.individual_roles["atoti_user"].add("ROLE_SHARE")

__NOTE:__ For basic authentication, __ROLE_USER__ is automatically assigned to users upon creation. However, it's not the case for users from authentication provider such as Auth0 or Google.  
We will have to grant the role corresponding to __ROLE_USER__ to the users in the authentication provider.

#### 3.3.2 Multiple roles assignment  

We grant the managers only access to the data available under their parent companies.  
These restrictions will be applied under the role __ATOTI_ROLE_INSPIRE__ and __ATOTI_ROLE_RESTAURANT__ respectively.  

Also, the managers will be granted __ROLE_SHARE__ for them to share the objects such as dashboards and widgets for which they are the owners of.

In [19]:
if auth_mode.value == AUTH_BASIC:
    user_service_client.individual_roles["Inspire_manager"].update(
        ["ROLE_SHARE", "ATOTI_ROLE_INSPIRE"]
    )
    user_service_client.individual_roles["Restaurant_manager"].update(
        ["ROLE_SHARE", "ATOTI_ROLE_RESTAURANT"]
    )

Each company has two users that have even more restricted access than the managers.  
User 1 of each company can only access data for restaurants of category _burgers_ with role __ATOTI_ROLE_BURGER__.  
User 2 of each company can only access data for restaurants of category _sandwich_ and _snack_ with roles __ATOTI_ROLE_SANDWICH__ and __ATOTI_ROLE_SNACK__.  

Combined with either the role __ATOTI_ROLE_INSPIRE__ or __ATOTI_ROLE_RESTAURANT__, they will only see the specific category of restaurants under their parent companies.

In [20]:
if auth_mode.value == AUTH_BASIC:
    user_service_client.individual_roles["Inspire_user1"].update(
        ["ATOTI_ROLE_INSPIRE", "ATOTI_ROLE_BURGER"]
    )
    user_service_client.individual_roles["Inspire_user2"].update(
        ["ATOTI_ROLE_INSPIRE", "ATOTI_ROLE_SANDWICH", "ATOTI_ROLE_SNACK"]
    )

    user_service_client.individual_roles["Restaurant_user1"].update(
        ["ATOTI_ROLE_RESTAURANT", "ATOTI_ROLE_BURGER"]
    )
    user_service_client.individual_roles["Restaurant_user2"].update(
        ["ATOTI_ROLE_RESTAURANT", "ATOTI_ROLE_SANDWICH", "ATOTI_ROLE_SNACK"]
    )

### 3.4. [For OIDC and LDAP only] Role assignment in atoti

The names in the authentication provider can be different from those in atoti.    
Roles are assigned to users in the authentication provider and restrictions applied based on this map.  

While a role from the authentication provider can be mapped to multiple atoti roles, it is easier to modify user's access by updating the roles in the authentication provider.  
It is a design consideration whether to have a one-to-one role map or to grant multiple atoti roles to an authentication provider role.  
E.g. The mapping `"atoti_user": ["ROLE_USER", "ROLE_SHARE"]` meant that all the users with __atoti user__ role assigned will be able to do sharing.  
Alternatively, it could be implemented in Auth0 as shown below, where individual users are granted the roles __atoti user__ and __ROLE_SHARE__.
<img src="https://data.atoti.io/notebooks/security/img/user_roles.png" width="50%" />

#### 2.4.1. Assign default roles to authenticated users

We could default the roles such as __ROLE_SHARE__ for all users who are logged in successfully.  
Depending on the authentication mechanism applied, use either [`user_service_client.oidc`](https://docs.atoti.io/latest/lib/atoti-plus/atoti_plus.user_service_client.oidc.security.html) or [`user_service_client.ldap`](https://docs.atoti.io/latest/lib/atoti-plus/atoti_plus.user_service_client.ldap.security.html) module to perform the operations.

In [21]:
if auth_mode.value in [AUTH_OIDC_AUTH0, AUTH_OIDC_GOOGLE]:
    user_service_client.oidc.default_roles.update(["ROLE_SHARE"])
elif auth_mode.value == AUTH_LDAP:
    user_service_client.ldap.default_roles.update(["ROLE_SHARE"])

__BE CAUTION__ when granting __ROLE_USER__ to users by default as users with this role will be able to access the application and the available data. It's better to grant __ROLE_USER__ to the users individually instead.

#### 3.4.2. Map roles between Authentication Provider and atoti  

The names in the authentication provider can be different from those in atoti.    
Roles are assigned to users in the authentication provider and associated to the atoti roles in the `role_mapping` below.

##### 3.4.2.1. Role mapping for OIDC (Auth0 example)

In [22]:
if auth_mode.value == AUTH_OIDC_AUTH0:
    print("Map roles from Auth0 to roles in atoti")
    user_service_client.oidc.role_mapping.update(
        {
            # atoti reserved roles
            "admin": ["ROLE_ADMIN"],
            "atoti_user": ["ROLE_USER"],
            "ROLE_SHARE": ["ROLE_SHARE"],
            # user-defined roles
            "ROLE_INSPIRE": [role_inspire.name],
            "ROLE_RESTAURANT": [role_restaurant.name],
            "ROLE_BURGER": [role_team_burger.name],
            "ROLE_SANDWICH": [role_team_sandwich.name],
            "ROLE_SNACK": [role_team_snack.name],
        }
    )

##### 3.4.2.2. Role mapping for OIDC (Google example)

__atoti is only able to use Google for authentication.__  
Roles are assigned to the users in atoti and they can be granted without having to restart the application.  

Instead of mapping the roles from Authentication Provider as in the case of Auth0, we use the user's email address to map to the assigned atoti roles.

In [23]:
if auth_mode.value == AUTH_OIDC_GOOGLE:
    print("Map roles from Google to roles in atoti")
    user_service_client.oidc.role_mapping.update(
        {
            "inspire_m@test.com": ["ROLE_USER", role_inspire.name],
            "inspire_user1@test.com": [
                "ROLE_USER",
                role_inspire.name,
                role_team_burger.name,
            ],
            "inspire_user2@test.com": [
                "ROLE_USER",
                role_inspire.name,
                role_team_sandwich.name,
                role_team_snack.name,
            ],
            "restaurant_m@test.com": ["ROLE_USER", role_restaurant.name],
            "restaurant_user1@test.com": [
                "ROLE_USER",
                role_restaurant.name,
                role_team_burger.name,
            ],
            "restaurant_user2@test.com": [
                "ROLE_USER",
                role_restaurant.name,
                role_team_sandwich.name,
                role_team_snack.name,
            ],
        }
    )

##### 3.4.2.3. Role mapping for LDAP

Similar to Auth0, we can map the roles downloaded from LDAP to the roles in atoti.  

The names in the authentication provider can be different from those in atoti.  
Roles are assigned to users in the authentication provider and associated to the atoti roles in the `role_mapping` below.  

Be mindful that regardless of the name used in LDAP, we have to __uppercase the LDAP role name__ in the mapping.

In [24]:
if auth_mode.value == AUTH_LDAP:
    print("Map roles from LDAP to roles in atoti")
    user_service_client.ldap.role_mapping.update(
        {
            # atoti reserved roles
            "ROLE_ATOTI_ADMIN": ["ROLE_ADMIN"],
            "ROLE_ATOTI_USER": ["ROLE_USER"],
            "ROLE_ATOTI_SHARE": ["ROLE_SHARE"],
            # user-defined roles
            "ROLE_INSPIRE": [role_inspire.name],
            "ROLE_RESTAURANT": [role_restaurant.name],
            "ROLE_BURGER": [role_team_burger.name],
            "ROLE_SANDWICH": [role_team_sandwich.name],
            "ROLE_SNACK": [role_team_snack.name],
        }
    )

## 4. Test login and access management in web application  

Try out any of these users (Password for users created with BasicSecurity are "password"):

___Administrator___
- atoti_admin (password: atoti_pwd)

___Generic user___
- atoti_user (password: atoti_pwd)

___Inspire Brands users___
- Inspire_user1 (Access for restaurants of category Burger only)
- Inspire_user2 (Access for restaurants of category Sandwich and Snack only)
- Inspire_manager

___Restaurant Brands International LLC users___
- Restaurant_user1 (Access for restaurants of category Burger only)
- Restaurant_user2 (Access for restaurants of category Sandwich and Snack only)
- Restaurant_manager

In [25]:
session.link()

Open the notebook in JupyterLab with the atoti extension enabled to see this link.

<div style="text-align: center;" ><a href="https://www.atoti.io/?utm_source=gallery&utm_content=security-implementation" target="_blank" rel="noopener noreferrer"><img src="https://data.atoti.io/notebooks/banners/discover-try.png" alt="Try atoti"></a></div>